In [ ]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from ISLP import load_data
from sklearn.preprocessing import StandardScaler
import pandas as pd

NYSE = load_data('NYSE')
cols = ['DJ_return', 'log_volume', 'log_volatility']

X = pd.DataFrame(
    StandardScaler().fit_transform(NYSE[cols]),
    columns=cols,
    index=NYSE.index
)

lags = 5
X_lagged = pd.concat([X.shift(i) for i in range(1, lags+1)], axis=1)
X_lagged.columns = [f"{col}_lag{i}" for i in range(1, lags+1) for col in cols]
X_lagged = X_lagged.dropna()
Y = X.loc[X_lagged.index, 'log_volume']

X_tensor = torch.tensor(X_lagged.values, dtype=torch.float32)
Y_tensor = torch.tensor(Y.values, dtype=torch.float32).unsqueeze(1)

train_size = int(0.8 * len(X_tensor))
X_train, X_test = X_tensor[:train_size], X_tensor[train_size:]
Y_train, Y_test = Y_tensor[:train_size], Y_tensor[train_size:]

train_ds = TensorDataset(X_train, Y_train)
test_ds = TensorDataset(X_test, Y_test)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=64)

In [ ]:
class NonlinearARModel(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )
    def forward(self, x):
        return self.net(x)

model = NonlinearARModel(X_train.shape[1])

In [ ]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 200

for epoch in range(epochs):
    model.train()
    for xb, yb in train_loader:
        optimizer.zero_grad()
        pred = model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()

In [ ]:
model.eval()
with torch.no_grad():
    Y_pred = model(X_test)
    ss_res = ((Y_test - Y_pred)**2).sum()
    ss_tot = ((Y_test - Y_test.mean())**2).sum()
    r2_test = 1 - ss_res/ss_tot

print(f"Test R² (nonlinear AR): {r2_test.item():.4f}")

Test R² (nonlinear AR): 0.3461


The nonlinear AR model trained on the flattened lagged sequences achieved a **test R² of 0.346**, which is slightly lower than the linear AR model’s R² of 0.391. This indicates that, in this case, the nonlinear feedforward network does not improve predictive performance over the simpler linear model. One possible reason is that the underlying relationships in the lagged NYSE data are largely linear, so the added complexity of a nonlinear network does not provide additional explanatory power and may even introduce slight overfitting. Overall, while nonlinear models are flexible and can capture complex patterns, for this dataset a **linear AR model appears sufficient** and more robust.